Julia code for Kitaev model.
\section{Kitaev model without symmetry}

In [339]:
function index(N1, N2, c, r, atom)
    #Calculate the index of the atoms in the (r,c) unit cell
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c 
    if atom == 'A'
        ind = 2 * n + 1
    elseif atom == 'B'
        ind = 2 * n + 1 + 1
    else
        return println("ERROR: The atom index should be A or B.")
    return ind
    end
end

function bits(tag::Int,i::Int, N1, N2)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
end
    
function StateConfig(tag, N1, N2)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
end

function flip(tag, i, j, N1, N2)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function KitaevRhom(J, N1, N2)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice
    Inputes: N1, N2: column and row numbers
             J: coupling constants
    Outpus: The Non-zero Hamiltonian element H[i] is in the 
            position (row[i], col[i])
            H, row, col : type: list =#
    SiteNum = N1 * N2 * 2
    dim = 2^SiteNum
    
    numNZero = 0  #count the number of non-zero element
    data = Float64[]
    row = Int[]
    col = Int[]
    for tag = 0: dim-1
        #println("The state numnber is: " , tag)
        temp = Dict{Int, Float64}() # A temporay dict
        
        for r = 0: N2-1
            for c = 0: N1-1
                ind = index(N1, N2, c, r, 'A')
                # Sx Sx
                next = index(N1, N2, c, r, 'B')
                
                ntag = flip(tag, ind, next, N1, N2)
                if ntag in keys(temp)
                    temp[ntag] += -J[1] /4
                else
                    temp[ntag] = -J[1] /4
                end  
                
                # Sy Sy
                next = index(N1, N2, c - 1, r, 'B')
                
                ntag = flip(tag, ind, next, N1, N2) 
                if ntag in keys(temp)
                    temp[ntag] += J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                else
                    temp[ntag] = J[2] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                end
                        
                #Sz Sz
                next = index(N1, N2, c, r - 1, 'B')
                    
                if tag in keys(temp)
                    temp[tag] += - J[3] * (bits(tag,ind, N1, N2) - 0.5) * (bits(tag,next, N1, N2) - 0.5)
                else
                    temp[tag] = - J[3] * (bits(tag,ind, N1, N2) - 0.5)* (bits(tag,next, N1, N2) - 0.5)
                end
            end
        end
                        
        odTemp = sort(collect(keys(temp)))
        for sf in odTemp
            append!(col, [tag + 1])
            append!(row, [sf + 1])
            append!(data,[temp[sf]])
            numNZero += 1
        end
    end
    
    println("non zero elemens number =" , numNZero)
    println("Percentage =" , numNZero/(dim * dim) * 100,"%")
    return col, row, data
end

function makeH(col, row, data)
    H = sparse(row, col, data)
    return H
end


makeH (generic function with 1 method)

In [455]:
using SparseArrays
using Arpack

N1 = 3
N2 = 3
J = [1,1,1]

@time col, row, data = KitaevRhom(J, N1, N2)
@time H =  makeH(col, row, data)


@time e,x = eigs(H)
println(e/(N1*N2*2))

non zero elemens number =4980736
Percentage =0.0072479248046875%
  2.290302 seconds (17.68 M allocations: 1.961 GiB, 24.62% gc time)
  0.218868 seconds (21 allocations: 158.001 MiB, 2.23% gc time)
  8.474366 seconds (2.59 k allocations: 62.103 MiB, 0.29% gc time)
[0.198493, -0.198493, -0.198493, 0.198493, -0.198493, 0.198493]


In [340]:
using SparseArrays
using Arpack

N1 = 3
N2 = 3
J = [1,1,1]

@time col, row, data = KitaevRhom(J, N1, N2)
@time H =  makeH(col, row, data)


@time e,x = eigs(H)
println(e/(N1*N2*2))

non zero elemens number =4980736
Percentage =0.0072479248046875%
  2.030082 seconds (17.67 M allocations: 1.960 GiB, 20.86% gc time)
  0.250455 seconds (2.19 k allocations: 158.101 MiB, 1.86% gc time)
  5.590781 seconds (1.76 k allocations: 62.072 MiB, 0.40% gc time)
[0.198493, -0.198493, -0.198493, 0.198493, 0.195481, -0.195481]


\section{Kitaev model with translation symmetry}

In [537]:
function index(N1::Int, N2::Int, c::Int, r::Int, atom)
    #Calculate the index of the atoms in the (r,c) unit cell
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c 
    if atom == 'A'
        ind = 2 * n + 1
    elseif atom == 'B'
        ind = 2 * n + 1 + 1
    else
        return println("ERROR: The atom index should be A or B.")
    return ind
    end
end

function bits(tag::Int,i::Int, N1, N2)
    # Chenck the i-th element of the binary representation of tag
    SiteNum  = N1 * N2 * 2
    mask = 2^(SiteNum - i)
    n = tag & mask
    if n == mask
        return 1
    else
        return 0
    end
end

function StateConfig(tag, N1, N2)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
end

function flip(tag, i, j, N1, N2)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function cyclebits(n1::Int,n2::Int, tag::Int, N1::Int, N2::Int)
    #=Performs a cyclic permutations of n1 steps to the right and n2 steps upward.
       Inputs: tag: tag of the reference state
               n1: cycle times of n1-direction, columns,if n1=0, no cycle in columns
               n2: cycle times of n2-direction, rows,if n2=0, no cycle in rows
               N1: number of columns
               N2: number of rows
       Outputs: ntag: the tag of the outcoming state=#
    SiteNum = N1 * N2 * 2
    #config = StateConfig(tag, N1, N2)
    #cfig = [parse(Int,x) for x in config]
    cfig = [x for x = 1: SiteNum]

    a = reshape(cfig, 2, N1, N2)
    b = circshift(a, (0, n1, n2))
    ncfig = reshape(b, (SiteNum, 1))
    #println(ncfig)
    ntag = 0
    for i in 1:SiteNum
        ntag += 2^(SiteNum - i) * bits(tag, ncfig[i], N1, N2)
    end  
    return ntag
end

function MomBasis(k1::Int, k2::Int, N1::Int, N2::Int)
    #=Momentum basis list of a given k1, k2
       Inputs: k1: momentum in n1-direction
               k2: momentum in n2-direction
               N1: number of columns
               N2: number of rows
       Outpus: mbasis: momentum basis list
               Ra: list of corresponding cycle period 
                   R1: n1-direction 
                   R2: n2-direction=#
    SiteNum = N1 * N2 * 2
    dim = 2^SiteNum
    
    basis = ones(Int, dim)
    
    m1 = Dict{Int, Int}()
    m2 = Dict{Int, Int}()
    mb = Int[]
    
    #println("Finish initialize")
    # Translation symmetry along n1-direction
    for tag = 0: (dim-1)
        if basis[tag + 1] == 1
            for n1= 1:N1
                ntag = cyclebits(n1, 0, tag, N1, N2)
                if ntag > tag
                    basis[ntag + 1] = 0
                elseif ntag == tag
                    basis[ntag + 1] = 0
                    if k1%(N1/n1) == 0
                        m1[tag] = n1
                    end
                    break
                end
            end
        end
    end
 
    # Translation symmetry along n2-direction
    basis = ones(Int, dim)
    for tag = 0: (dim-1)
        if basis[tag + 1] == 1
            for n2= 1:N2
                ntag = cyclebits(0, n2, tag, N1, N2)
                if ntag > tag
                    basis[ntag + 1] = 0
                elseif ntag == tag
                    basis[ntag + 1] = 0
                    if k2%(N2/n2) == 0
                        m2[tag] = n2
                    end
                    break
                end
            end
        end
    end
   
    for x in collect(keys(m1))
        if x in collect(keys(m2))
            append!(mb,[x])
        end
    end
    return mb, m1, m2
end

MomBasis (generic function with 1 method)

In [548]:
N1 = 3
N2 = 3

k1 = 0
k2 = 0

@time mb, m1, m2 = MomBasis(k1, k2, N1, N2)
println("Finished!")

110.652508 seconds (8.17 M allocations: 57.434 GiB, 6.28% gc time)
Finished!


In [545]:
using Profile
@profile mb, m1, m2 = MomBasis(k1, k2, N1, N2)
println("Finished!")

Finished!


In [546]:
Profile.print()

519 ./task.jl:259; (::getfield(IJulia, Symbol("##15#1...
 519 ...yLI42/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  519 ./essentials.jl:741; invokelatest
   519 ./essentials.jl:742; #invokelatest#1
    518 ...c/execute_request.jl:67; execute_request(::ZMQ.Socket, ::...
     1   ...c/SoftGlobalScope.jl:209; softscope_include_string(::Modu...
     1   ...c/SoftGlobalScope.jl:211; softscope_include_string(::Modu...
      1 ./none:0; (::getfield(Base.Meta, Symbol("#k...
       1 ./meta.jl:123; #parse#1(::Bool, ::Bool, ::Bool...
        1 ./logging.jl:491; with_logger(::Function, ::Base....
         1 ./logging.jl:395; with_logstate(::getfield(Base....
          1 ./meta.jl:124; (::getfield(Base.Meta, Symbol(...
           1 ./essentials.jl:355; cconvert(::Type, ::Int64)
     3   ...c/SoftGlobalScope.jl:217; softscope_include_string(::Modu...
      1 ...c/SoftGlobalScope.jl:194; add_linenum(::Expr, ::Int64, ::S...
       1 ...c/SoftGlobalScope.jl:187; shift_linenum(::Expr, ::Int64, ...
   

        2 ./In[523]:68; cyclebits(::Int64, ::Int64, ::I...
         2 ./abstractarraymath.jl:184; circshift
          2 ./multidimensional.jl:933; circshift!(::Array{Int64,3}, :...
           2 ./multidimensional.jl:962; _circshift!
            1 ./multidimensional.jl:957; _circshift!(::Array{Int64,3}...
             1 ./int.jl:54; mod
        2 ./In[523]:73; cyclebits(::Int64, ::Int64, ::I...
        1 ./reshapedarray.jl:51; cyclebits(::Int64, ::Int64, ::I...
       31  ./In[523]:136; MomBasis(::Int64, ::Int64, ::I...
        4  ./abstractdict.jl:98; keys
         4 ./abstractdict.jl:40; Type
          4 ./abstractdict.jl:40; Type
        25 ./array.jl:544; collect
         25 ./array.jl:550; _collect
          1  ./abstractarray.jl:0; copyto!(::Array{Int64,1}, ::B...
          1  ./abstractarray.jl:668; copyto!(::Array{Int64,1}, ::B...
           1 ./dict.jl:683; iterate
            1 ./dict.jl:685; iterate
             1 ./array.jl:729; getindex
          23 ./abstractarray.jl:672; 

In [496]:
m1 = Dict{Int, Int}()
m2 = Dict{Int, Int}()
m1[2] = 5
m1[1] = 6
println(m1)
m2[2] = 5
m2[7] = 8
println(m2)

Dict(2=>5,1=>6)
Dict(7=>8,2=>5)


In [454]:
N1 = 3
N2 = 2

k1 = 0
k2 = 0

@time mb, r1, r2 = MomBasis(k1, k2, N1, N2)
println("Finished!")

  0.195224 seconds (243.32 k allocations: 132.265 MiB, 52.48% gc time)
Finished!


In [505]:
for x in collect(keys(m1))
    if x in collect(keys(m2))
        println(x)
    end
end

2


In [536]:
N1 = 2
N2 = 3
tag = 167
n1 = 2
n2 = 3
ntag = cyclebits(n1,n2, tag, N1, N2)
print(ntag)

167

In [485]:
println(m2)

Dict(7=>8,2=>5)
